## Building Project From Existing Layer

In [ ]:
# import libraries
import arcgis
import datetime

# Set some properties
org = "https://arcgis.com"
username = "<username>"
project_id = "<project-id>"
restaurants_fs_id = "<feature-service-id>"

gis = arcgis.gis.GIS(org, username)
project_data = gis.content.get(project_id).get_data()

### Adding The Restaurants Layer To The Dispatcher Map

In [ ]:
dispatcher_webmap = arcgis.mapping.WebMap(gis.content.get(project_data["dispatcherWebMapId"]))
restaurants_layer = gis.content.get(restaurants_fs_id).layers[0]
dispatcher_webmap.layers
dispatcher_webmap.add_layer(restaurants_layer)
dispatcher_webmap.update({})
dispatcher_webmap

### Adding A Worker/Inspector To The Project

In [ ]:
worker = arcgis.features.Feature(attributes={
    "name": "<worker name>",
    "status": 0,
    "userId": "<worker_username>"
})
workers_fl = gis.content.get(project_data["workers"]["serviceItemId"]).layers[0]
workers_fl.edit_features(adds=[worker])

In [ ]:
group = gis.groups.get(project_data["groupId"])
group.add_users(["<worker_username>"])
worker = workers_fl.query(where="userId='<worker_username>'").features[0]

### Add An "Kitchen Inspection" Assignment Type

In [ ]:
assignment_fl = gis.content.get(project_data["assignments"]["serviceItemId"]).layers[0]
assignment_types = ["Kitchen Inspection"]
for field in assignment_fl.properties["fields"]:
    if field["name"].lower() == "assignmenttype":
        for assignment_type in assignment_types:
            # Next coded value
            codes = [x["code"] for x in field["domain"]["codedValues"]]
            if codes:
                next_code = max(codes) + 1
            else:
                next_code = 1
            field["domain"]["codedValues"].append(
                {
                    "name": assignment_type,
                    "code": next_code
                }
            )
        break
assignment_fl.manager.update_definition({
    'fields': assignment_fl.properties['fields']
})

### Add an assignment to each restaurant

In [ ]:
restaurants = restaurants_layer.query().features
assignments = []
for restaurant in restaurants:
    new_assignment = arcgis.features.Feature(geometry=restaurant.geometry, attributes={
        "location": restaurant.attributes["Address"],
        "assignmentType": 1,
        "assignmentRead": 0,
        "workerId": worker.attributes["OBJECTID"],
        "status": 1,
        "assignedDate": datetime.datetime.now()
    })
    assignments.append(new_assignment)
assignment_fl.edit_features(adds=assignments)

In [ ]:
dispatcher_webmap